In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [4]:
#load the model
model=load_model('model.h5')

#load encoder and scalar
with open('one_hot_encoding_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [6]:
# sample input data

input_data={
    'CreditScore': 600,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 10000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [10]:
#one hot encoding for Geography
geo_encoded=label_encoder_geo.transform([[input_data['Geography']]]).toarray()

geo_encoded_df=pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\ADMIN\Desktop\Customer_churning\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [15]:
input_df=pd.DataFrame([input_data])

In [17]:
input_data=pd.concat([input_df.reset_index(drop=True),geo_encoded_df],axis=1)

In [18]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,10000,2,1,1,50000


In [21]:
#concatination with one hot encoded
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)


KeyError: "['Geography'] not found in axis"

In [22]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,10000,2,1,1,50000,1.0,0.0,0.0


In [23]:
#scaling input data
input_scaled=scaler.transform(input_df)
input_scaled


array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -1.05836066,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [25]:
#Predict churn
prediction=model.predict(input_scaled)

1/1 [==============================] - 0s 32ms/step


In [27]:
prediction_proba=prediction[0][0]
prediction_proba

0.020298615

In [29]:
if prediction_proba>0.5:
    print('Customer likely to churn')
else:
    print("Customer not likely to  churn")    

Customer not likely to  churn
